In [2]:
import pandas as pd
import re

In [5]:
# Set display option to show all rows
pd.set_option('display.max_rows', None)

### Step 1: Concat all vehicle files into one single file.

In [4]:
# read all scrapped vehicle csv files
df_Toyota = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Toyota.csv")
df_Ford = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Ford.csv")
df_Benz = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Benz.csv")
df_Hyundai = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Hyundai.csv")
df_Isuzu = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Isuzu.csv")
df_Mazda = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Mazda.csv")
df_Suzuki = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Suzuki.csv")
df_Mitsubishi = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Mitsubishi.csv")
df_Peugeot = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Peugeot.csv")
df_Others = pd.read_csv(r"C:\Users\BullWang\Documents\GitHub\Brenda-s-Repo\Pre-Owned Vehicles Project\Vehicle scrapped data\vehicle_Others.csv")

# combine all the dataframe into one csv
df_vehicle = pd.concat([df_Toyota, df_Ford, df_Benz, df_Hyundai, df_Isuzu,
                        df_Mazda, df_Suzuki, df_Mitsubishi, df_Peugeot, df_Others])

### Step 2: Observe the dataframe

In [7]:
df_vehicle.head(5)

,Unnamed: 0,Make,Model,Price,View,Availability,Year,Odometer,Fuel Type,Drive train,...,Seats,Engine,Series,Body,Colour,VIN,Badge,Drive Type,Stock No.,Rego
0,0,Toyota,Landcruiser,"129,990",1330,In stock,2022,"28,680kms",Diesel,4WD,...,5 seats,6 cylinder 3.3L,FJA300R,4 door 5 seat Wagon,Grey,JTMAAABJ204020112,LC300 Sahara ZX (4x4),4WD,198365,1HSL522
1,1,Toyota,Landcruiser,"129,988",263,In stock,2021,"48,828kms",Diesel,4WD,...,7 seats,8 cylinder 4.5L,VDJ200R,4 door 7 seat Wagon,Crystal Pearl,JTMHV05JX04328010,LC200 Sahara (4x4),4WD,197425,1IEV822
2,2,Toyota,Landcruiser 70 Series,"127,988",910,In stock,2023,"5,050kms",Diesel,AWD,...,5 seats,8 cylinder 4.5L,Vdjl79R,4 door 5 seat Double Cab Chassis,Sandy Taupe,JTEBVL1J00B011339,LC79 GXL,AWD,198293,1HZK664
3,3,Toyota,Landcruiser,"126,999",1247,In stock,2021,"56,267kms",Diesel,4WD,...,7 seats,8 cylinder 4.5L,VDJ200R,4 door 7 seat Wagon,White,JTMHV05J504332949,LC200 Sahara (4x4),4WD,196112,1HHW267
4,4,Toyota,Landcruiser,"125,988",557,In stock,2021,"73,204kms",Diesel,4WD,...,7 seats,8 cylinder 4.5L,VDJ200R,4 door 7 seat Wagon,Crystal Pearl,JTMHV05J604332703,LC200 Sahara (4x4),4WD,196855,XB952G


In [40]:
df_vehicle.dtypes

Make                           object
Model                          object
Price                          object
View                            int64
Availability                   object
Year                            int64
Odometer                        int32
Fuel Type                      object
Drive train                    object
Transmission                   object
Seats                           Int64
Engine                         object
Series                         object
Body                           object
Colour                         object
VIN                            object
Stock No.                       int64
Rego                           object
Engine Displacement (Litre)    object
Engine Cylinders               object
Body Type                      object
Door                           object
dtype: object

The above dataframe requires the following cleaning:
1. Drop unnecessary columns: Unnamed_0 index column, drive type column (dupliate), and badge column.
2. Remove unit info. in data: the Odometer column (kms) and Seat column (seats)
3. Split columns containing additional information: the Engine (cylinders; Litre) and Body column (door; seat; body type)

### Step 3: Drop unnecessary columns

In [8]:
# drop the Unname:0 column and create a new index
df_vehicle.drop(columns=['Unnamed: 0'], inplace = True)
df_vehicle.reset_index(drop=True, inplace=True)

In [9]:
# drop the drive type column, as it is a duplicate column of drive train
df_vehicle.drop(columns='Drive Type', inplace=True)

In [10]:
# drop the badge column, as the analysis will not be using this info.
df_vehicle.drop(columns='Badge', inplace=True)

### Step 4: Remove unnecessary unit content 

In [11]:
# clean the odometer column: replace 'kms' and ',' and change the typr to integer
df_vehicle['Odometer'] = df_vehicle['Odometer'].str.replace('kms', '', regex= False).str.replace(',', '', regex = False).astype(int)

In [12]:
# clean the seat colum: extract only numeric values from the string 
df_vehicle['Seats'] =  df_vehicle['Seats'].str.extract('(\d+)', expand=False)
# convert numeric values into integer type while leaving the NaN values
df_vehicle['Seats'] = pd.to_numeric(df_vehicle['Seats'], errors='coerce').astype('Int64')

### Step 5: Split information stored in the same column 

In [13]:
# clean the Engine column: separate the displacement values (litres) and the number of cylinders into different columns
# create a 'Engine_Displacement' column; extract the displacement form the engine column 
df_vehicle['Engine Displacement (Litre)'] = df_vehicle['Engine'].str.extract(r'([\d.]+)(?=L)') # remove L using lookhead assersion

# Define the function to extract the number of cylinders
def extract_cylinder_num(engine):
    if isinstance(engine, str):
        parts = engine.split(' ')
        if 'cylinder' in engine:
            cylinder_index = parts.index('cylinder')
            return parts[cylinder_index -1]
        else:
            return 0
    else:
        return 0
# create a 'Engine_Cylinders' column; split the string and return the first
df_vehicle['Engine Cylinders'] = df_vehicle['Engine'].apply(extract_cylinder_num)

In [14]:
# clean the Body column: Extract body types
# Define the function to extract the body type
def extract_body_type(body):
    if isinstance(body, str):
        parts = body.split(' ')
        if 'seat' in parts:
            seat_index = parts.index('seat')
            if seat_index + 1 < len(parts):
                return ' '.join(parts[seat_index + 1:])
            else:
                return ''  # Return empty if 'seat' is the last word. (e.g., '4 door 5 seat')
        elif 'door' in parts:
            door_index = parts.index('door')
            if door_index + 1 < len(parts):
                return ' '.join(parts[door_index + 1:])
            else:
                return ''  # Return empty if 'door' is the last word. (e.g., '4 door')
        else:
            return body  # Return the entire string if it’s a standalone vehicle type. (e.g., 'Van')
    return ''
# create a 'Body Type' column   
df_vehicle['Body Type'] = df_vehicle['Body'].apply(extract_body_type)

In [15]:
# clean the Body column: Extract the number of doors
def extract_door_num(body):
    if isinstance(body, str):
        parts = body.split(' ')
        if 'door' in parts:
            part_index = parts.index('door')
            return parts[part_index -1]
        else:
            return ''
    else:
        return ''
# create a 'Door' column  
df_vehicle['Door'] = df_vehicle['Body'].apply(extract_door_num)

### Step 6: Replace missing data with appropriate values

In [35]:
# check for missing values in all columns
missing_valiues = df_vehicle.isnull().sum()
missing_valiues

Make                            0
Model                           0
Price                           0
View                            0
Availability                    0
Year                            0
Odometer                        0
Fuel Type                      70
Drive train                     4
Transmission                    3
Seats                           8
Engine                          0
Series                         15
Body                            6
Colour                          0
VIN                             0
Stock No.                       0
Rego                            1
Engine Displacement (Litre)     1
Engine Cylinders                0
Body Type                       0
Door                            0
dtype: int64

In [25]:
# Set display option to show all rows
pd.set_option('display.max_columns', None)

In [37]:
df_vehicle[df_vehicle['Fuel Type'].isnull()].head(3)
df_vehicle[df_vehicle['Series'].isnull()].head(3)
df_vehicle[df_vehicle['Drive train'].isnull()].head(3)
df_vehicle[df_vehicle['Seats'].isnull()].head(3)
df_vehicle[df_vehicle['Transmission'].isnull()].head(3)
df_vehicle[df_vehicle['Engine Displacement (Litre)'].isnull()].head(3)
df_vehicle[df_vehicle['Body'].isnull()].head(3)

,Make,Model,Price,View,Availability,Year,Odometer,Fuel Type,Drive train,Transmission,Seats,Engine,Series,Body,Colour,VIN,Stock No.,Rego,Engine Displacement (Litre),Engine Cylinders,Body Type,Door
1479,Harley-Davidson,Breakout,"25,969",211,In stock,2014,7419,NaN,RWD,NaN,<NA>,2 cylinder 1.7L,MY14,NaN,Radioactive Green,5HD1BFVG7FB018443,197360,1JS193,1.7,2,,
1514,Ducati,Panigale V2,"22,999",618,In stock,2023,900,NaN,RWD,Manual,<NA>,2 cylinder 1L,MY23,NaN,Red,ZDM1H00AAPB011454,197973,1KP052,1,2,,
1523,Harley-Davidson,Street Bob 114,"22,969",19,In stock,2021,4388,NaN,NaN,NaN,<NA>,nullL,(FXBBS),NaN,Black,5HD1YYK61MB031479,199716,1JV456,NaN,0,,


In [43]:
# replace null cells with 'unkown' in the all columns
df_vehicle['Fuel Type'].fillna(' ')
df_vehicle['Drive train'].fillna(' ')
df_vehicle['Transmission'].fillna(' ')
df_vehicle['Seats'].fillna(0)
df_vehicle['Series'].fillna(' ')
df_vehicle['Body'].fillna('0')
df_vehicle['Engine Displacement (Litre)'].fillna('0')
df_vehicle['Engine Cylinders'].fillna('0')

0       6
1       8
2       8
3       8
4       8
5       8
6       6
7       6
8       8
9       8
10      8
11      8
12      8
13      8
14      8
15      8
16      8
17      8
18      8
19      8
20      6
21      8
22      8
23      8
24      8
25      8
26      8
27      8
28      8
29      8
30      8
31      8
32      8
33      8
34      8
35      8
36      8
37      8
38      8
39      8
40      8
41      8
42      8
43      8
44      8
45      8
46      8
47      8
48      8
49      8
50      6
51      8
52      8
53      8
54      8
55      8
56      8
57      8
58      8
59      8
60      8
61      8
62      8
63      8
64      8
65      8
66      8
67      8
68      8
69      8
70      8
71      8
72      8
73      8
74      8
75      8
76      8
77      8
78      8
79      8
80      8
81      8
82      8
83      8
84      8
85      8
86      8
87      8
88      8
89      8
90      8
91      8
92      8
93      8
94      8
95      8
96      8
97      8
98      8
99      8


### Step 7: Change data type 

In [44]:
df_vehicle.dtypes

Make                           object
Model                          object
Price                          object
View                            int64
Availability                   object
Year                            int64
Odometer                        int32
Fuel Type                      object
Drive train                    object
Transmission                   object
Seats                           Int64
Engine                         object
Series                         object
Body                           object
Colour                         object
VIN                            object
Stock No.                       int64
Rego                           object
Engine Displacement (Litre)    object
Engine Cylinders               object
Body Type                      object
Door                           object
dtype: object

In [46]:
# Change Seats, Engine_Displacement (Litre), and Engine_Cylinders to numeric data types
df_vehicle['Seats'].astype('Int64')
df_vehicle['Engine Displacement (Litre)'].astype(float)
df_vehicle['Engine Cylinders'].astype(int)

0       6
1       8
2       8
3       8
4       8
5       8
6       6
7       6
8       8
9       8
10      8
11      8
12      8
13      8
14      8
15      8
16      8
17      8
18      8
19      8
20      6
21      8
22      8
23      8
24      8
25      8
26      8
27      8
28      8
29      8
30      8
31      8
32      8
33      8
34      8
35      8
36      8
37      8
38      8
39      8
40      8
41      8
42      8
43      8
44      8
45      8
46      8
47      8
48      8
49      8
50      6
51      8
52      8
53      8
54      8
55      8
56      8
57      8
58      8
59      8
60      8
61      8
62      8
63      8
64      8
65      8
66      8
67      8
68      8
69      8
70      8
71      8
72      8
73      8
74      8
75      8
76      8
77      8
78      8
79      8
80      8
81      8
82      8
83      8
84      8
85      8
86      8
87      8
88      8
89      8
90      8
91      8
92      8
93      8
94      8
95      8
96      8
97      8
98      8
99      8


### Final Step: Write the dataframe to a csv file

In [47]:
df_vehicle.to_csv("Pre_Owned Vehicle data_cleaned.csv")